## Decision Tree
### Our Goal:
To diagnostically predict whether a patient has diabetes.

### To Explore:
1. DecisionTreeClassifier
2. BaggingClassifier
3. AdaBoostClassifier
4. RandomForestClassifier
    

In [2]:
#General libraries needed
import numpy as np
import pandas as pd

#For Decision Tree implementation
from scipy.stats import entropy
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier

#For Bagging implementation
from sklearn.ensemble import BaggingClassifier

#For AdaBoost implementation
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier


# Import train_test_split function
from sklearn.model_selection import train_test_split 
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics 

In [3]:
col_names = ['pregnant', 'glucose', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age', 'label']
# load dataset
df = pd.read_csv("Diabetes.csv", header = None, names=col_names)
df.head()

,pregnant,glucose,bp,skin,insulin,bmi,pedigree,age,label
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
#split dataset in features and target variable
#label is the result we trying to predict [1 = got diabetes, 0 = healthy]

feature_cols = ['pregnant', 'insulin', 'bmi', 'age','glucose','bp','pedigree']
X = df[feature_cols]          # Features, independent var
y = df.label                  # Target variable, dependent var (result)

In [5]:
# Split dataset into training set and test set
# Random partitions

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3) # 70% training and 30% test

In [6]:
#Create tree
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7316017316017316


In [7]:
#printing the full tree

from sklearn import tree
from sklearn.tree import export_graphviz
tree.export_graphviz(clf, out_file='tree.dot', feature_names=feature_cols) #produces dot file

import pydot

(graph,) = pydot.graph_from_dot_file('tree.dot')
graph.write_png('full_tree.png')

In [8]:
#Pruning! basically stop the tree before it max. 
#Directly implement the DecisionTreeClassifier on the training set. To ensure pruning, we set the max_depth=4.
#Prevents overfitting.. but how do we do know when is too much????

dptree = DecisionTreeClassifier(max_depth=4)
dptree.fit(X_train, y_train)

#Test!
# Get the predicted y array

y_pred = dptree.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7186147186147186


In [9]:
# Create Decision Tree classifer object
# possible parameters to pass in and default values:
# SEE: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

# criterion = "gini" or "entropy"
# max_depth: int (default, none -- go as deep)
# min_samples_split: int (default 2), if use float, it will consider the value as a proportion -- relative to dataset I believe)

for depth in range(1, 20):
    
    pred_sum = 0
    
    trials = 100
    
    for trial in range(0, trials):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3) # 70% training and 30% test
        
        decision_tree_test = DecisionTreeClassifier(
            max_depth = depth,
        )

        dt = decision_tree_test.fit(X_train,y_train)

        dt_pred = dt.predict(X_test)
        pred_sum += metrics.accuracy_score(y_test, dt_pred)
    
    print("Depth = ", depth , "Average Accuracy:", pred_sum/trials)

Depth =  1 Average Accuracy: 0.7197402597402598
Depth =  2 Average Accuracy: 0.73987012987013
Depth =  3 Average Accuracy: 0.7386147186147184
Depth =  4 Average Accuracy: 0.7290043290043289
Depth =  5 Average Accuracy: 0.7370995670995673
Depth =  6 Average Accuracy: 0.7258874458874456
Depth =  7 Average Accuracy: 0.715108225108225
Depth =  8 Average Accuracy: 0.7084848484848486
Depth =  9 Average Accuracy: 0.7123809523809522
Depth =  10 Average Accuracy: 0.7065367965367968
Depth =  11 Average Accuracy: 0.7020779220779222
Depth =  12 Average Accuracy: 0.7004761904761905
Depth =  13 Average Accuracy: 0.7006493506493505
Depth =  14 Average Accuracy: 0.7002597402597401
Depth =  15 Average Accuracy: 0.7054978354978357
Depth =  16 Average Accuracy: 0.7051948051948052
Depth =  17 Average Accuracy: 0.7040259740259743
Depth =  18 Average Accuracy: 0.6992640692640693
Depth =  19 Average Accuracy: 0.697402597402597


<h1> Bagging (with Decision Tree) </h1>

In [10]:
#Create the Bagging classifier. Default base classifiers is Decision Tree. 
# - n_estimator is the number of base classifiers (number of trees trained) (i.e. weak learners)

model = BaggingClassifier(n_estimators=200)

#Fit the training feature Xs and training label Ys
model.fit(X_train, y_train)

#Use the trained model to predict the test data
y_pred = model.predict(X_test)

# Find the confusion matrix of the result
cm = metrics.confusion_matrix(y_pred, y_test)
print(cm)

[[127  32]
 [ 22  50]]


In [11]:
# Find the accuracy of the result

bagged_results = metrics.accuracy_score(y_pred, y_test)
print(bagged_results)

0.7662337662337663


<h1>AdaBoost (with Decision Tree) </h1>

*Note that the default AdaBoost implementation in SKLearn is Decision Tree

In [12]:
#Create the AdaBoost classifier. Default base classifiers is Decision Tree. 
# - n_estimator is the number of base classifiers (i.e. weak learners)
# - learning_rate controls the weight adjustments of each base classifiers. Default is 1

model = AdaBoostClassifier(n_estimators=200, learning_rate = 0.1)  #if you change learning_rates/ tune the no of weak base classifier, the 
#final accuracy will change

#Fit the training feature Xs and training label Ys
model.fit(X_train, y_train)

#SVC classifer takes long time to run BUT it actually gives a very high accuracy

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=0.1, n_estimators=200, random_state=None)

In [13]:
#Use the trained model to predict the test data
y_pred = model.predict(X_test)

# Find the confusion matrix of the result
cm = metrics.confusion_matrix(y_pred, y_test)
print(cm)

# Find the accuracy of the result
asr = metrics.accuracy_score(y_pred, y_test)
print(asr)

[[123  41]
 [ 26  41]]
0.70995670995671


In [14]:
#The bigger the learning rate the more emphathsis on older trees

for learningrate in range(1, 8):
    
    pred_sum = 0
    
    trials = 20
    
    for trial in range(0, trials):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3) # 70% training and 30% test
        
        decision_tree_test = AdaBoostClassifier(n_estimators=200, learning_rate = 0.3 ** learningrate)

        dt = decision_tree_test.fit(X_train,y_train)

        dt_pred = dt.predict(X_test)
        
        pred_sum += metrics.accuracy_score(y_test, dt_pred)
    
    print("learningrate = ", 0.3 ** learningrate)
    print("Average Accuracy:", pred_sum/trials)
    print("-----------------------------------")

learningrate =  0.3
Average Accuracy: 0.7545454545454545
-----------------------------------
learningrate =  0.09
Average Accuracy: 0.7642857142857143
-----------------------------------
learningrate =  0.026999999999999996
Average Accuracy: 0.7595238095238095
-----------------------------------
learningrate =  0.0081
Average Accuracy: 0.7467532467532466
-----------------------------------
learningrate =  0.0024299999999999994
Average Accuracy: 0.7361471861471862
-----------------------------------
learningrate =  0.0007289999999999998
Average Accuracy: 0.7277056277056275
-----------------------------------
learningrate =  0.00021869999999999995
Average Accuracy: 0.7183982683982684
-----------------------------------


# <h1> Random Forest </h1>

In [15]:
#Create the Random Forest classifier.
#n_estimator is the number of base classifiers (i.e. weak learners) number of trees you have
#default is sqrt of the m var

model = RandomForestClassifier(n_estimators=200)
model.fit(X_train,y_train)
y_pred=model.predict(X_test)

#SVC classifer takes long time to run BUT it actually gives a very high accuracy

print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7316017316017316


In [16]:
#put a trail loop inside 
for power in range(7):
    
    n_estimators = 5 ** power

    random_forest_test = RandomForestClassifier(
        n_estimators = n_estimators,
        bootstrap = True,
        n_jobs = -1
    )

    random_forest_test.fit(X_train, y_train)
    y_pred = random_forest_test.predict(X_test)
    print("Accuracy for n_estimators:", n_estimators, " is ", metrics.accuracy_score(y_test, y_pred))

Accuracy for n_estimators: 1  is  0.7272727272727273
Accuracy for n_estimators: 5  is  0.6926406926406926
Accuracy for n_estimators: 25  is  0.7402597402597403
Accuracy for n_estimators: 125  is  0.7229437229437229
Accuracy for n_estimators: 625  is  0.7445887445887446
Accuracy for n_estimators: 3125  is  0.7402597402597403
Accuracy for n_estimators: 15625  is  0.7359307359307359


Adapted from your Seniors DA (The Pioneer Batch) : **Ding Yang, Linus Cheng, Tan Kin Meng, and Kaelyn** for these codes, which they used in their DAP sharing :)